In [ ]:
import pandas as pd
import numpy as np
import datetime
import io
import os
import glob
import sys
from googleads import adwords
from datetime import datetime, timedelta


output= io.StringIO()

adwords_client = adwords.AdWordsClient.LoadFromStorage('xxx.yaml')

adwords_client.SetClientCustomerId('xxx')

report_downloader = adwords_client.GetReportDownloader(version='v201809')

#https://developers.google.com/adwords/api/docs/appendix/reports/url-performance-report
#https://developers.google.com/adwords/api/docs/guides/awql?hl=ru

report_query = (adwords.ReportQueryBuilder()
                  .Select('CampaignId', 'CampaignName','AdGroupId','AdGroupName','Domain','Url','Device',
                          'Impressions','Clicks','Interactions','Cost','Conversions')
                  .From('URL_PERFORMANCE_REPORT')
                  .Where('CampaignStatus').In('ENABLED')
                  .During('LAST_7_DAYS') 
                  .Build())

report_downloader.DownloadReportWithAwql(report_query, 'CSV', output, skip_report_header=True,
          skip_column_header=False, skip_report_summary=True,
          include_zero_impressions=False)

output.seek(0)

types= { 'CampaignId':pd.np.int64, 'Clicks': pd.np.float64, 'Impressions': pd.np.float64,
         'Cost': pd.np.float64,'Conversions': pd.np.float64,'ConversionValue': pd.np.float64 }

df = pd.read_csv(output,low_memory=False, dtype= types, na_values=[' --'])

df['Cost'] = df['Cost'] / 1000000

In [ ]:
df

In [ ]:
df_KMS = df[~df['Domain'].str.contains('mobileapp')].sort_values('Impressions',ascending = False)
df_MA = df[df['Domain'].str.contains('mobileapp')].sort_values('Impressions',ascending = False)

In [ ]:
ma_list = []
for e in list(df_MA['Domain']):
    ma_list.append(e.split('mobileapp::')[1])
    
pd.Series(list(set(ma_list))).to_excel('MobApp.xlsx',index=False)

In [ ]:
df_MA.to_excel('MobApp.xlsx',index=False)